In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # ضع هنا مسار النموذج المدرب على كرة القدم

# تحميل الصورة
image = cv2.imread("frame.jpg")

# تشغيل الكشف
results = model(image)[0]

# استخراج الكائنات
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)
    
    # نفترض أن الكلاس 0 هو لاعب، والكلاس 1 هو كرة (حسب التدريب الخاص بك)
    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# نفرض أن الفريق المهاجم على اليمين، والتسلل يتم بناءً على موضع الـ x
attacking_team = []
defending_team = []

# تصنيف الفرق بناءً على الموقع الأفقي (يمكن تحسينه لاحقاً بالألوان أو المعرفات)
median_x = np.median([p['center'][0] for p in players])
for p in players:
    if p['center'][0] > median_x:
        attacking_team.append(p)
    else:
        defending_team.append(p)

# تحديد آخر مدافع (أقصى قيمة x في الفريق المدافع)
last_defender_x = max([p['center'][0] for p in defending_team])

# تحديد اللاعبين المتسللين
offside_players = []
for p in attacking_team:
    if p['center'][0] > last_defender_x:
        offside_players.append(p)

# رسم النتائج
for p in players:
    x1, y1, x2, y2 = p['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

for p in offside_players:
    x1, y1, x2, y2 = p['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم خط التسلل
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة الناتجة
cv2.imwrite("result.jpg", image)


0: 384x640 6 persons, 263.7ms
Speed: 7.1ms preprocess, 263.7ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)


True

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # غيّر المسار إلى النموذج المدرب على كرة القدم

# تحميل الصورة
image = cv2.imread("frame.jpg")

# تشغيل النموذج على الصورة
results = model(image)[0]

# استخراج الكائنات: لاعبين والكرة
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)

    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# تقسيم الفريقين بشكل تقريبي حسب المواقع (يمين ويسار)
attacking_team = []
defending_team = []

if len(players) >= 4:
    median_x = np.median([p['center'][0] for p in players])
    for p in players:
        if p['center'][0] > median_x:
            attacking_team.append(p)
        else:
            defending_team.append(p)
else:
    print("عدد اللاعبين قليل جداً لتحديد الفريقين.")
    attacking_team = players  # مؤقتاً نضع الكل كمهاجمين

# حساب موقع آخر مدافع
if defending_team:
    last_defender_x = max([p['center'][0] for p in defending_team])
else:
    last_defender_x = image.shape[1] // 2  # وسط الصورة كبديل

# تحديد أقرب لاعب مهاجم للكرة (يُفترض أنه المستقبل)
potential_offside_player = None
if ball and attacking_team:
    ball_center_x = (ball['bbox'][0] + ball['bbox'][2]) // 2
    ball_center_y = (ball['bbox'][1] + ball['bbox'][3]) // 2

    min_dist = float('inf')
    for p in attacking_team:
        px, py = p['center']
        dist = np.sqrt((px - ball_center_x)**2 + (py - ball_center_y)**2)
        if dist < min_dist:
            min_dist = dist
            potential_offside_player = p

# التحقق إذا كان هذا اللاعب متسلل
if potential_offside_player and potential_offside_player['center'][0] > last_defender_x:
    x1, y1, x2, y2 = potential_offside_player['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم جميع اللاعبين (للتوضيح فقط)
#for p in players:
   # x1, y1, x2, y2 = p['bbox']
   # cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

# رسم خط التسلل
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة الناتجة
cv2.imwrite("result1.jpg", image)
print("تم حفظ الصورة باسم result1.jpg")


0: 384x640 6 persons, 492.7ms
Speed: 35.0ms preprocess, 492.7ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)
تم حفظ الصورة باسم result1.jpg


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # استبدل هذا بمسار النموذج المدرب على كرة القدم

# تحميل الصورة
image = cv2.imread("frame.jpg")

# تشغيل النموذج على الصورة
results = model(image)[0]

# استخراج الكائنات: لاعبين والكرة
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)

    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# تقدير الفريقين
attacking_team = []
defending_team = []

if len(players) >= 4:
    median_x = np.median([p['center'][0] for p in players])
    for p in players:
        if p['center'][0] > median_x:
            attacking_team.append(p)
        else:
            defending_team.append(p)
else:
    attacking_team = players  # مؤقتاً نعتبر الكل مهاجمين

# تحديد آخر مدافع
if defending_team:
    last_defender_x = max([p['center'][0] for p in defending_team])
else:
    last_defender_x = image.shape[1] // 2

# تحديد اللاعب المتسلل بناءً على التمرير
potential_offside_player = None
if ball and attacking_team:
    ball_center_x = (ball['bbox'][0] + ball['bbox'][2]) // 2
    ball_center_y = (ball['bbox'][1] + ball['bbox'][3]) // 2

    min_dist = float('inf')
    for p in attacking_team:
        px, py = p['center']
        dist = np.sqrt((px - ball_center_x)**2 + (py - ball_center_y)**2)
        if dist < min_dist:
            min_dist = dist
            potential_offside_player = p

# التحقق من التسلل
if potential_offside_player and potential_offside_player['center'][0] > last_defender_x:
    x1, y1, x2, y2 = potential_offside_player['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم خط التسلل فقط
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة
cv2.imwrite("result2.jpg", image)
print("تم حفظ الصورة باسم result2.jpg")


0: 384x640 6 persons, 277.0ms
Speed: 15.7ms preprocess, 277.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
تم حفظ الصورة باسم result2.jpg


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # غيّر هذا لموقع النموذج المدرب الخاص بك

# تحميل الصورة
image = cv2.imread("frame.jpg")

# تشغيل النموذج على الصورة
results = model(image)[0]

# استخراج الكائنات: لاعبين والكرة
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)

    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# تقدير الفريقين بناءً على الموقع الأفقي للاعبين
attacking_team = []
defending_team = []

if len(players) >= 4:
    median_x = np.median([p['center'][0] for p in players])
    for p in players:
        if p['center'][0] > median_x:
            attacking_team.append(p)
        else:
            defending_team.append(p)
else:
    attacking_team = players  # إذا كان عدد اللاعبين قليل، نعتبر الكل مهاجمين

# تحديد آخر مدافع
if defending_team:
    last_defender_x = max([p['center'][0] for p in defending_team])
else:
    last_defender_x = image.shape[1] // 2  # تحديد خط المنتصف كبديل إذا لم يوجد مدافعين

# تحديد اللاعب المتسلل بناءً على أقرب مسافة للكرة
potential_offside_player = None
if ball and attacking_team:
    ball_center_x = (ball['bbox'][0] + ball['bbox'][2]) // 2
    ball_center_y = (ball['bbox'][1] + ball['bbox'][3]) // 2

    min_dist = float('inf')
    for p in attacking_team:
        px, py = p['center']
        dist = np.sqrt((px - ball_center_x)**2 + (py - ball_center_y)**2)
        if dist < min_dist:
            min_dist = dist
            potential_offside_player = p

# التحقق من التسلل
if potential_offside_player and potential_offside_player['center'][0] > last_defender_x:
    x1, y1, x2, y2 = potential_offside_player['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم خط التسلل
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة
cv2.imwrite("result3.jpg", image)
print("تم حفظ الصورة باسم result3.jpg")


0: 384x640 6 persons, 445.0ms
Speed: 54.5ms preprocess, 445.0ms inference, 39.4ms postprocess per image at shape (1, 3, 384, 640)
تم حفظ الصورة باسم result3.jpg


In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # غيّر هذا لموقع النموذج المدرب الخاص بك

# تحميل الصورة
image = cv2.imread("frame.jpg")

# تشغيل النموذج على الصورة
results = model(image)[0]

# استخراج الكائنات: لاعبين والكرة
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)

    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# تقدير الفريقين بناءً على الموقع الأفقي للاعبين
attacking_team = []
defending_team = []

if len(players) >= 4:
    median_x = np.median([p['center'][0] for p in players])
    for p in players:
        if p['center'][0] > median_x:
            attacking_team.append(p)
        else:
            defending_team.append(p)
else:
    attacking_team = players  # إذا كان عدد اللاعبين قليل، نعتبر الكل مهاجمين

# تحديد آخر مدافع
if defending_team:
    last_defender_x = max([p['center'][0] for p in defending_team])
else:
    last_defender_x = image.shape[1] // 2  # تحديد خط المنتصف كبديل إذا لم يوجد مدافعين

# تحديد اللاعب المتسلل بناءً على أقرب مسافة للكرة
potential_offside_player = None
if ball and attacking_team:
    ball_center_x = (ball['bbox'][0] + ball['bbox'][2]) // 2
    ball_center_y = (ball['bbox'][1] + ball['bbox'][3]) // 2

    min_dist = float('inf')
    for p in attacking_team:
        px, py = p['center']
        dist = np.sqrt((px - ball_center_x)**2 + (py - ball_center_y)**2)
        if dist < min_dist:
            min_dist = dist
            potential_offside_player = p

# التحقق من التسلل
if potential_offside_player and potential_offside_player['center'][0] > last_defender_x:
    x1, y1, x2, y2 = potential_offside_player['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم خط التسلل
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة
cv2.imwrite("result4.jpg", image)
print("تم حفظ الصورة باسم result4.jpg")


0: 384x640 6 persons, 228.1ms
Speed: 9.1ms preprocess, 228.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
تم حفظ الصورة باسم result4.jpg


In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # غيّر هذا لموقع النموذج المدرب الخاص بك

# تحميل الصورة
image = cv2.imread("frame1.jpg")

# تشغيل النموذج على الصورة
results = model(image)[0]

# استخراج الكائنات: لاعبين والكرة
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)

    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# تقدير الفريقين بناءً على الموقع الأفقي للاعبين
attacking_team = []
defending_team = []

if len(players) >= 4:
    median_x = np.median([p['center'][0] for p in players])
    for p in players:
        if p['center'][0] > median_x:
            attacking_team.append(p)
        else:
            defending_team.append(p)
else:
    attacking_team = players  # إذا كان عدد اللاعبين قليل، نعتبر الكل مهاجمين

# تحديد آخر مدافع
if defending_team:
    last_defender_x = max([p['center'][0] for p in defending_team])
else:
    last_defender_x = image.shape[1] // 2  # تحديد خط المنتصف كبديل إذا لم يوجد مدافعين

# تحديد اللاعب المتسلل بناءً على أقرب مسافة للكرة
potential_offside_player = None
if ball and attacking_team:
    ball_center_x = (ball['bbox'][0] + ball['bbox'][2]) // 2
    ball_center_y = (ball['bbox'][1] + ball['bbox'][3]) // 2

    min_dist = float('inf')
    for p in attacking_team:
        px, py = p['center']
        dist = np.sqrt((px - ball_center_x)**2 + (py - ball_center_y)**2)
        if dist < min_dist:
            min_dist = dist
            potential_offside_player = p

# التحقق من التسلل
if potential_offside_player and potential_offside_player['center'][0] > last_defender_x:
    x1, y1, x2, y2 = potential_offside_player['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم خط التسلل
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة
cv2.imwrite("result5.jpg", image)
print("تم حفظ الصورة باسم result4.jpg")


0: 288x640 14 persons, 1 sports ball, 305.4ms
Speed: 39.1ms preprocess, 305.4ms inference, 16.0ms postprocess per image at shape (1, 3, 288, 640)
تم حفظ الصورة باسم result4.jpg


In [10]:
import cv2
import numpy as np
from ultralytics import YOLO

# تحميل النموذج المدرب
model = YOLO("yolov8n.pt")  # ضع هنا مسار النموذج المدرب على كرة القدم

# تحميل الصورة
image = cv2.imread("frame1.jpg")

# تشغيل الكشف
results = model(image)[0]

# استخراج الكائنات
players = []
ball = None

for result in results.boxes.data.tolist():
    x1, y1, x2, y2, conf, cls = result
    cls = int(cls)
    
    # نفترض أن الكلاس 0 هو لاعب، والكلاس 1 هو كرة (حسب التدريب الخاص بك)
    if cls == 0:  # لاعب
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        players.append({'bbox': (int(x1), int(y1), int(x2), int(y2)), 'center': (center_x, center_y)})
    elif cls == 1:  # كرة
        ball = {'bbox': (int(x1), int(y1), int(x2), int(y2))}

# نفرض أن الفريق المهاجم على اليمين، والتسلل يتم بناءً على موضع الـ x
attacking_team = []
defending_team = []

# تصنيف الفرق بناءً على الموقع الأفقي (يمكن تحسينه لاحقاً بالألوان أو المعرفات)
median_x = np.median([p['center'][0] for p in players])
for p in players:
    if p['center'][0] > median_x:
        attacking_team.append(p)
    else:
        defending_team.append(p)

# تحديد آخر مدافع (أقصى قيمة x في الفريق المدافع)
last_defender_x = max([p['center'][0] for p in defending_team])

# تحديد اللاعبين المتسللين
offside_players = []
for p in attacking_team:
    if p['center'][0] > last_defender_x:
        offside_players.append(p)

# رسم النتائج
for p in players:
    x1, y1, x2, y2 = p['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

for p in offside_players:
    x1, y1, x2, y2 = p['bbox']
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
    cv2.putText(image, "OFFSIDE", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# رسم خط التسلل
cv2.line(image, (last_defender_x, 0), (last_defender_x, image.shape[0]), (0, 255, 255), 2)

# حفظ الصورة الناتجة
cv2.imwrite("result6.jpg", image)


0: 288x640 14 persons, 1 sports ball, 304.6ms
Speed: 6.5ms preprocess, 304.6ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)


True

In [11]:
import cv2
from ultralytics import YOLO

# تحميل نموذج YOLOv8
model = YOLO("yolov8n.pt")

# فتح الفيديو
video_path = "offside_example.MP4"  
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("offside_result_1.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # تحليل الفريم
    results = model(frame)[0]
    annotated_frame = results.plot()

    # جمع مواقع اللاعبين
    players = []
    for box in results.boxes:
        if int(box.cls[0]) == 0:  # class 0 = person
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x = (x1 + x2) // 2
            players.append((center_x, x1, y1, x2, y2))

    # رسم خط التسلل
    if len(players) >= 2:
        players.sort(key=lambda p: p[0])  # حسب موقعهم الأفقي
        offside_line_x = players[-2][0]  # ثاني آخر مدافع
        cv2.line(annotated_frame, (offside_line_x, 0), (offside_line_x, height), (0, 255, 255), 2)

        # تحديد اللاعب المتسلل
        for player in players:
            center_x, x1, y1, x2, y2 = player
            if center_x > offside_line_x + 10:
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(annotated_frame, "Offside", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    out.write(annotated_frame)

cap.release()
out.release()
print("تم حفظ الفيديو الناتج باسم: offside_result_1.mp4")


0: 352x640 15 persons, 772.3ms
Speed: 140.1ms preprocess, 772.3ms inference, 68.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 15 persons, 186.6ms
Speed: 8.0ms preprocess, 186.6ms inference, 3.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 15 persons, 256.0ms
Speed: 10.6ms preprocess, 256.0ms inference, 2.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 16 persons, 235.7ms
Speed: 7.2ms preprocess, 235.7ms inference, 3.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 persons, 279.9ms
Speed: 21.7ms preprocess, 279.9ms inference, 2.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 15 persons, 254.1ms
Speed: 10.3ms preprocess, 254.1ms inference, 3.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 persons, 184.6ms
Speed: 6.7ms preprocess, 184.6ms inference, 2.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 persons, 389.1ms
Speed: 14.2ms preprocess, 389.1ms inference, 2.3ms p